### Introduction
In this notebook we will test out our custom quantized backprop code, and benchmark it against autograd, which is an automatic differentiation Python library.

In [3]:
import autograd.numpy as np

from autograd import grad

from mlp4autograd import mlp4autograd


In [4]:
def error(params):
    # inference error wrapper that we will use with autograd

    ref.infer(x_in, params)

    return ref.error(y_truth)

def ref_v_autograd_error(ref_grad, autograd_grad):
    # calculates mean percentage error of two calculated gradients, usually custom Python vs autograd

    ref_grad_w = ref_grad[0]
    ref_grad_b = ref_grad[1]

    autograd_grad_w = autograd_grad[0]
    autograd_grad_b = autograd_grad[1]

    N = len(ref_grad_w)

    delta_w = []
    delta_b = []

    for i in range(N):

        delta_w.append(np.average(np.absolute(np.divide(ref_grad_w[i] - autograd_grad_w[i], autograd_grad_w[i]))))

        delta_b.append(np.average(np.absolute(np.divide(ref_grad_b[i] - autograd_grad_b[i], autograd_grad_b[i]))))

    return (np.average(delta_w) + np.average(delta_b))/2

First we create an instant of our custom backprop module with floating point precision. We can use this to verify our implementation of the backprop algorithm.

In [5]:
ref = mlp4autograd()

# no quantization, [3,7,7,5] architecture
# initialize weights uniformly in [0,1]
ref.set_quant(quant = False)
ref.net_shape([3,7,7,5])
ref.init_params(mode="uniform")
params = [ref._weight, ref._bias]

# generate some random input
x_in = np.random.random((3,1))
ref.infer(x_in, params)

# let the truth be x_in with some small perturbation
# we want some small difference so gradient is nonzero but not too large
y_truth = ref._activation[-1] + np.random.random((5,1))

# calculate gradient according to truth
ref.grad(y_truth, False)

Next we calculate the gradient using the autograd library

In [6]:
# create the gradient function
gradient = grad(error)

# evaluate the gradient function using the same set of parameters
grad_eval = gradient(params)

Finally, we can compare our own gradient calculations with floating point precision against autograd's gradient. We expect exactly 0% error, because the calculations we use are exactly the same as autograd.

In [7]:
print("Percentage difference: " + "{:2.2%}".format(ref_v_autograd_error(ref.params_grad, grad_eval)))

Percentage difference: 0.00%


Next, we create a instance of the backprop module with (24,12) quantization. We will need to quantize the parameters, inputs, and outputs. Then we will evaluate the gradient with fixed point arithmetic.

In [8]:
D_BITS = 24
Q_BITS = 12

# (24,12) quantization, [3,7,7,5] architecture
ref_q = mlp4autograd()
ref_q.set_quant(D_BITS, Q_BITS, True)
ref_q.net_shape([3,7,7,5])

# quantize weights and biases
weight_q = []
bias_q = []

for w in ref._weight:
    weight_q.append(np.round(w * (2 ** Q_BITS)))

for b in ref._bias:
    bias_q.append(np.round(b * (2 ** Q_BITS)))

ref_q._weight = weight_q
ref_q._bias = bias_q

# quantize inputs and outputs
x_in_q = np.round(x_in * (2 ** Q_BITS))
y_truth_q = np.round(y_truth * (2 ** Q_BITS))

# perform inference and backprop
ref_q.infer(x_in_q, [weight_q, bias_q])
ref_q.grad(y_truth_q)

# de-quantize (fixed point back to floating point)
weight_grad_q = []
bias_grad_q = []

for wg in ref_q._weight_dif:
    weight_grad_q.append(np.divide(wg, (2 ** Q_BITS)))

for bg in ref_q._bias_dif:
    bias_grad_q.append(np.divide(bg, (2 ** Q_BITS)))

param_grad_q = [weight_grad_q, bias_grad_q]

Finally, we can calculate the mean percent error between fixed point gradient and autograd gradient. We should see mean percent errors less than 1 %.

In [9]:
print("Percentage difference: " + "{:2.2%}".format(ref_v_autograd_error(param_grad_q, grad_eval)))

Percentage difference: 0.52%


In [12]:
ref._weight[0]

array([[0.75619983, 0.40104644, 0.60145777],
       [0.53197353, 0.06374854, 0.018671  ],
       [0.40614024, 0.49858538, 0.94812328],
       [0.13445347, 0.60993221, 0.61763712],
       [0.77910687, 0.24742395, 0.14752764],
       [0.23340611, 0.3955668 , 0.94701494],
       [0.42314411, 0.35262293, 0.508145  ]])

In [24]:

# importing the csv module
import csv
# field names
fields = ['Name', 'Branch', 'Year', 'CGPA']
# data rows of csv file
rows = [['Nikhil', 'COE', '2', '9.0'],
        ['Sanchit', 'COE', '2', '9.1'],
        ['Aditya', 'IT', '2', '9.3'],
        ['Sagar', 'SE', '1', '9.5'],
        ['Prateek', 'MCE', '3', '7.8'],
        ['Sahil', 'EP', '2', '9.1']]
# name of csv file
filename = "university_records.csv"
# writing to csv file
with open(filename, 'w', newline = '') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the fields
    csvwriter.writerow(fields,)
    # writing the data rows
    csvwriter.writerows(ref._weight[0])

    csv_reader = csv.reader(csvfile)

    csvreader = csv.reader(csvfile)
 
    csvfile.close()

with open(filename, 'r') as csvfile:

    csvreader = csv.reader(csvfile)

    rows = []

    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:

        conv = []

        for f in row:
            conv.append(float(f))

        rows.append(conv)

    np.array(rows)

print(rows)



[[0.7561998343493976, 0.4010464419530745, 0.6014577714595644], [0.5319735287264586, 0.06374854004239738, 0.01867100036760616], [0.40614024329483356, 0.49858537934207514, 0.948123280108787], [0.13445346874151376, 0.6099322058160056, 0.6176371212367342], [0.7791068747014938, 0.24742395085913382, 0.1475276361049399], [0.23340611402675526, 0.39556679662526995, 0.9470149415453747], [0.42314411103616956, 0.3526229322762674, 0.5081449987359857]]
